In [ ]:
from nbdev import *

In [ ]:
%nbdev_default_export app

Cells will be exported to emmaus_walking.app,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Emmaus Walking Streamlit App

> This is the front-end application built in Streamlit (which I believe is not compatible with Jupyter notebooks).
> However `nbdev` converts this to a straight .py script by default - which should be possible to `streamlit run`.

In [ ]:
%nbdev_hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
%nbdev_export
import numpy as np
import pandas as pd
import datetime as dt
import streamlit as st
from streamlit_folium import folium_static
import folium
import os, io
import activityio as aio
from IPython.display import display
from dateutil.parser import parse
#import matplotlib


#from pandas_profiling import ProfileReport
#from streamlit_pandas_profiling import st_profile_report

2020-08-16 14:44:45.391 WARNING streamlit.config_option: 
════════════════════════════════════════════════
global.logLevel IS DEPRECATED.
global.logLevel has been replaced with logger.level

This option will be removed on or after 2020-11-30.

Please update /Users/mjboothaus/.streamlit/config.toml.
════════════════════════════════════════════════



In [ ]:
%nbdev_export
DATA_INFO = 'Health Fit / Apple Watch (Author)'
AUTHOR_INFO = 'AUTHOR: [Michael J. Booth](https://about.me/mjboothaus)'
APP_NAME = 'Emmaus Walking Mapping App'

st.beta_set_page_config(page_title=APP_NAME)

In [ ]:
%nbdev_export

# TODO: use st.cache() and also look to pre-load and cache/feather data (or similar) - NB: use of @st.cache() below didn't work
def load_walk_data(walk_name):
    FIT_FILE_PATH = '/Users/mjboothaus/iCloud/Data/HealthFit/'
    data_dir = FIT_FILE_PATH + walk_name[0:3] + '/'
    data_files = [file for file in os.listdir(data_dir) if file.endswith('.fit')]
    walk_files = sorted(data_files)

    walk_data = []
    walk_date = []

    for iFile, file in enumerate(walk_files):
        walk_data.append(pd.DataFrame(aio.read(data_dir + file)))
        walk_date.append(parse(file[0:17]))
    return walk_data, walk_date, walk_files


def calc_walk_stats(walk_data):
    total_time = dt.timedelta(0)
    total_distance = 0

    for iHike, hike in enumerate(walk_data):
        total_time += hike.index.max()
        # print(iHike+1, walk_date[iHike], hike.index.max(), hike['dist'].max() / 1e3)
        total_distance += hike['dist'].max()
    total_distance /= 1e3

    start_coord = walk_data[0][['lat', 'lon']].iloc[0].tolist()
    end_coord = walk_data[-1][['lat', 'lon']].iloc[-1].tolist()
    return total_time, total_distance, start_coord, end_coord


def plot_walk(walk_df, map_handle, walk_colour, freq=100):
    points = []
    count = 0
    for index, row in walk_df.iterrows():
        count+=1
        if count%freq == 0:
            points.append((row['lat'], row['lon']))
            folium.PolyLine(points, color=walk_colour, weight=6).add_to(map_handle)


def plot_entire_walk(walk_data, map_handle):
    for iHike, hike in enumerate(walk_data):
        plot_walk(hike, map_handle, 'yellow')

In [ ]:
# walk_data, walk_date, walk_files = load_walk_data('B2M')

In [ ]:
#TODO: Need to add some tests!

In [ ]:
%nbdev_export
class SideBar:
    datasource = DATA_INFO
    datasize = 0   # look to calculate this (in MB?) - TEST: Comment change
    author = AUTHOR_INFO
    data_title = 'Data details...'
    data_local = False
    start_date = dt.date.today()
    end_date = dt.date.today()
    selected_data = None
    walk_name = ''


def app_sidebar(APP_NAME):
    WALK_NAME = ['B2M: Bondi to Manly', 'GNW: Great North Walk', 'GWW: Great West Walk']
    # WALK_SUBDIR_NAME = ['GNW', 'GWW', 'B2M']
    sb = SideBar()
    st.sidebar.info(APP_NAME)
    st.sidebar.markdown(sb.author)
    st.sidebar.markdown(sb.datasource)
    st.sidebar.info(sb.data_title)
    #st.sidebar.markdown('Datasize: ' + str(sb.datasize))
    sb.walk_name = st.sidebar.selectbox('Choose a walk', WALK_NAME, 0)
    return sb

In [ ]:
%nbdev_export
def app_mainscreen(APP_NAME, sb):
    
    #st.title(APP_NAME)
    st.header(sb.walk_name)
    # Load walking data
    walk_data, walk_date, walk_files = load_walk_data(sb.walk_name)
    total_time, total_distance, start_coord, end_coord = calc_walk_stats(walk_data)

    map_handle = folium.Map(start_coord, zoom_start=13, detect_retina=True, control_scale=True)
    plot_entire_walk(walk_data, map_handle)
    map_handle.fit_bounds(map_handle.get_bounds())

    #TODO: Change the following to .format() and .join() not string "addition"

    st.write('Total time: ' + str(total_time))
    st.write('Total distance: ' + str(total_distance))

    folium_static(map_handle, width=800, height=650)
    return map_handle, walk_data, walk_date

In [ ]:
%nbdev_export
def notebook_mainscreen(APP_NAME, sb):
    print(APP_NAME)

    # Load walking data
    walk_data, walk_date, walk_files = load_walk_data(sb.walk_name)
    total_time, total_distance, start_coord, end_coord = calc_walk_stats(walk_data)

    map_handle = folium.Map(start_coord, zoom_start=13, detect_retina=True, control_scale=True)
    plot_entire_walk(walk_data, map_handle)
    map_handle.fit_bounds(map_handle.get_bounds())

    print(sb.walk_name)
    print('Total time: ', total_time)
    print('Total distance: ', total_distance)

    #folium_static(map_handle)
    return map_handle, walk_data, walk_date

In [ ]:
%nbdev_export

sb = app_sidebar(APP_NAME)
app_mainscreen(APP_NAME, sb)

#map_handle, walk_data, walk_date = notebook_mainscreen(APP_NAME, sb)

Emmaus Walking Mapping App
B2M: Bondi to Manly
Total time:  0 days 12:49:36
Total distance:  47.932550000000006


In [ ]:
map_handle

In [ ]:
%nbdev_export
# TODOs:
#
# Setup drop-down to choose the specific walk to be mapped

In [ ]:
# Folium example of adding tooltips

# add marker for Opera House
    #tooltip = "Sydney Opera House"
    #folium.Marker(
    #    [-33.85719805, 151.21512338473752], popup="Sydney Opera House", tooltip=tooltip
    #).add_to(m)

    # call to render Folium map in Streamlit